# Evaluating an Information System

---

To evaluate an information system we need:
- text corpus
- set of information need - queries or classes
- relevance metric for each of the information needs


### Metrics for Information System Evaluation
----
#### Precision and Recall
![https://towardsdatascience.com/precision-and-recall-made-simple-afb5e098970f](https://miro.medium.com/max/828/1*xMl_wkMt42Hy8i84zs2WGg.png)

----
#### Fᵦ score

A factor indicating **how much more important recall is than precision**. For example, if we consider recall to be twice as important as precision, we can set β to 2. The standard F-score is equivalent to setting β to one.

![https://en.wikipedia.org/wiki/F-score](https://i.stack.imgur.com/swW0x.png)

### Example texts

In [1]:
text1 = '''
    Ние виждаме една серия от управленски провали. Укрепено ли е правителството - не, то е несъществуващо. Правителството е несъществуващото - в този тежък момент, когато достигаме близо 1000 заразени на ден, ако говорим за здравния проблем.

    Протестът на хората е супер основателен, защото те виждат провала на властта в двете важни сфери - икономиката и здравето.

    Това заяви евродепутатът от ДСБ/ЕНП Радан Кънев пред бТВ.

    По думите му на 15 октомври е трябвало България да представи пред Европейската комисия проект за национален възстановителен план, което "ще определени икономическата съдба на поколения българи - това са парите за преодоляване на COVID кризата, но и парите за адаптация на Зелената сделка и парите, с които икономиката ни да компенсира минимум едно 10-годишно изоставане от европейските икономически политики".

    Преди да бъде представен пред Комисията, нали трябва да бъде представен пред обществото, пред бизнеса, синдикатите. Това е проект, който ще определи дали ние ще позволим да бъдем последни или ще си повярваме, че можем да бъдем богата държава, богато общество и че можем да имаме модерна икономика, добави той.

    Според него няма дебат с европейските партньори, с политици и с частния финансов сектор.
    '''

text2 = '''
    Най-малко 34 души от афганистанските сили за сигурност, сред които и висш полицейски служител, са убити при нападение на талибаните в афганистанската провинция Тахар, съобщиха властите, цитирани от ДПА.

    Според полицията други 8 са били убити през нощта в друга част на страната, с което жертвите стават 42.

    Това е второто нападение на бунтовниците за последния месец, въпреи започналите в Катар миналия месец мирни преговори между правителството и талибаните. Припомняме, че за да стартират разговорите правителството освободи всички близо 5000 задържани талибани. След като условието беше изпълнено, на 12 септември преговорите за прекратяването на почти 20-годишната война започнаха.'''

text3 = '''
    ДОМ НА КИНОТО

    18:00 >> След огромния успех снощи при премиерата на дигитално възстановената първа серия на „МЕРА СПОРЕДЪ МЕРА“ в присъствието на режисьора Георги Дюлгеров, актьорите Руси Чанев и Стефан Мавродиев, оператора Радослав Спасов, художника Георги Тодоров-Жози и други членове на екипа, ви представяме втората част на филма - с напълно реставрирани картина и звук. Легендарната творба е създадена през 1981 година по сценарий на Руси Чанев и Георги Дюлгеров, по романа
    на Свобода Бъчварова „Литургия за Илинден“, режисирана от Дюлгеров и е едно от най-мащабните, епични и значими произведения на българското кино. В него присъстват както реални исторически личности, така и персонажи, родени от въображението на авторите с техните човешки драми, тревоги, съмнения.

    Разказът е обединен от метаморфозата на главния герой Дилбер Танас – от първичен овчар, част от патриархалната задруга до личност с индивидуално съзнание. Катализатор на тази промяна са борбите за независимост на македонските българи от 1901 до 1912 година. Съдбата на Дилбер Танас е метафора на общата ни история, в която човешки драми и политически игри променят изначалната идея на борбата и нейното значение. Това е първият български филм, пресъздаващ историята на Илинденско-Преображенското въстание през 1903 година, с участието на реални исторически личности - Апостол войвода, Христо Чернопеев, Яне Сандански, Пейо Яворов, Гоце Делчев, Георги Мучитан.

    Реставрираната визия, реализирана в Доли медия студио, е резултат от сканирането на целия негатив и неговото цялостно почистване; след това той е разчетен, направени са нови цветови корекции и е коригирана експозицията на кадрите, като са добавени визуални ефекти за подсилване на картината и подобряване на качеството й. Филмът е с изцяло нов звук – верен на оригинала, но преформатиран от моно в стерео 5+1, с добавени звукови ефекти. За да бъде улеснено възприемането на историята от младите поколения, при запазена оригинална фонограма със специфичните наречия и архаичната лексика, са добавени субтитри на литературен български език. '''

# Load train data

In [2]:
from nltk.corpus import PlaintextCorpusReader
import math
from helper_funcs import split_corpus, bernoulli_classifier

In [3]:
corpus_root = 'JOURNALISM.BG/C-MassMedia'
my_corpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
fileNames = my_corpus.fileids()

classesSet = set( [ file[:file.find('/')] for file in fileNames ] )
classes = sorted(list(classesSet - {'Z','D-Society'}))

print(classes)

['C-Culture', 'E-Economy', 'J-Politics', 'S-Military']


In [4]:
def get_words(file):
    words = my_corpus.words(file)
    return words

In [ ]:
full_class_corpus = [ [ get_words(file) for file in fileNames if file.find(c+'/')==0 ] for c in classes ]
full_class_corpus

In [6]:
test_class_corpus, train_class_corpus = split_corpus.split_class_corpus(full_class_corpus)

# Bernoulli Classifier

In [7]:
condProbB, priorB, VB =  bernoulli_classifier.train_bernoulli_NB(train_class_corpus)
initialCondProbB =  bernoulli_classifier.calc_initial_cond_prob(condProbB)

[--------------------------------------------------]


In [8]:
print('The first text is classified as: '+classes[
    bernoulli_classifier.apply_bernoulli_NB_SLOW(priorB, condProbB, text1.split())])
print('The second text is classified as: '+classes[
    bernoulli_classifier.apply_bernoulli_NB_SLOW(priorB, condProbB, text2.split())])
print('The third text is classified as: '+classes[
    bernoulli_classifier.apply_bernoulli_NB_SLOW(priorB, condProbB, text3.split())])

The first text is classified as: J-Politics
The second text is classified as: J-Politics
The third text is classified as: S-Military


In [9]:
print('The first text is classified as: '+classes[
    bernoulli_classifier.apply_bernoulli_NB(priorB, condProbB, initialCondProbB, text1.split())])
print('The second text is classified as: '+classes[
    bernoulli_classifier.apply_bernoulli_NB(priorB, condProbB, initialCondProbB, text1.split())])
print('The third text is classified as: '+classes[
      bernoulli_classifier.apply_bernoulli_NB(priorB, condProbB, initialCondProbB, text1.split())])

The first text is classified as: J-Politics
The second text is classified as: J-Politics
The third text is classified as: J-Politics


# Multinomial Classifier

In [10]:
from helper_funcs import multinomial_classifier

In [11]:
condProbM, priorM, VM = multinomial_classifier.train_multinomial_NB(train_class_corpus)

[--------------------------------------------------]


In [12]:
print('The first text is classified as: '+classes[
    multinomial_classifier.apply_multinomial_NB(priorM, condProbM, text1.split())])
print('The second text is classified as: '+classes[
    multinomial_classifier.apply_multinomial_NB(priorM, condProbM, text2.split())])
print('The third text is classified as: '+classes[
    multinomial_classifier.apply_multinomial_NB(priorM, condProbM, text3.split())])

The first text is classified as: J-Politics
The second text is classified as: S-Military
The third text is classified as: C-Culture


# Test

In [13]:
from helper_funcs import test_classifier

## Bernoulli Classifier

In [14]:
gamma = lambda text : bernoulli_classifier.apply_bernoulli_NB(priorB, condProbB, initialCondProbB, text)
confusionMatrix, precision, recall, Fscore, P, R, F1 = test_classifier.test_classifier(test_class_corpus, gamma)
print('Confusion matrix: ')
for row in confusionMatrix:
    for val in row:
        print('{:4}'.format(val), end = '')
    print()
print('Precision: '+str(precision))
print('Recall: '+str(recall))
print('F-scire: '+str(Fscore))
print('Overall precision: '+str(P)+', recall: '+str(R)+', F-score: '+str(F1))
print()

[--------------------------------------------------]
Confusion matrix: 
  42   1   2   0
   0  26  38   1
   0  19 700  16
   0   4  54 100
Precision: [1.0, 0.52, 0.8816120906801007, 0.8547008547008547]
Recall: [0.9333333333333333, 0.4, 0.9523809523809523, 0.6329113924050633]
F-scire: [0.9655172413793104, 0.45217391304347826, 0.9156311314584696, 0.7272727272727273]
Overall precision: 0.8592498720763799, recall: 0.8654037886340977, F-score: 0.8623158511395672



## Multinomial Classifier

In [15]:
gamma = lambda text : multinomial_classifier.apply_multinomial_NB(priorM, condProbM, text)
confusionMatrix, precision, recall, Fscore, P, R, F1 = test_classifier.test_classifier(test_class_corpus, gamma)
print('Confusion matrix: ')
for row in confusionMatrix:
    for val in row:
        print('{:4}'.format(val), end = '')
    print()
print('Precision: '+str(precision))
print('Recall: '+str(recall))
print('F-scire: '+str(Fscore))
print('Overall precision: '+str(P)+', recall: '+str(R)+', F-score: '+str(F1))
print()

[--------------------------------------------------]
Confusion matrix: 
  42   1   2   0
   0  40  24   1
   0  14 692  29
   0   1  21 136
Precision: [1.0, 0.7142857142857143, 0.9364005412719891, 0.8192771084337349]
Recall: [0.9333333333333333, 0.6153846153846154, 0.9414965986394558, 0.8607594936708861]
F-scire: [0.9655172413793104, 0.6611570247933884, 0.9389416553595656, 0.8395061728395061]
Overall precision: 0.9064095238245398, recall: 0.9072781655034895, F-score: 0.9068436366518053



**Further reading:**

[Text Classification using Naive Bayes - University of Edinburgh](https://www.inf.ed.ac.uk/teaching/courses/inf2b/learnnotes/inf2b-learn07-notes-nup.pdf)